# ECE523 - Homework 2 
* Solution by [Gregory Ditzler](http://gditzler.github.io)

In [1]:
import numpy as np 
import matplotlib.pylab as plt

# Solutions 

In [ ]:
def gen_cb(N:int=1000, a:float=0.5, alpha:float=0): 
    d = np.random.rand(N, 2).T
    d_transformed = np.array([d[0]*np.cos(alpha)-d[1]*np.sin(alpha), 
                              d[0]*np.sin(alpha)+d[1]*np.cos(alpha)]).T
    s = np.ceil(d_transformed[:,0]/a)+np.floor(d_transformed[:,1]/a)
    lab = 2 - (s%2)
    data = d.T
    return data, lab 

# test the function 
X, y = gen_cb()
plt.figure()
plt.scatter(X[y==1., 0], X[y==1., 1], color='r')
plt.scatter(X[y==2., 0], X[y==2., 1], color='b')

In [ ]:
def gen_gaussian(Nc:int=500, mu1=[-1,-1], mu2=[1,1], c1=[[1., -.25],[-.25, 1.]], c2=[[1., -.25],[-.25, 1.]]): 
    X1 = np.random.multivariate_normal(mean=mu1, cov=c1, size=Nc)
    X2 = np.random.multivariate_normal(mean=mu2, cov=c2, size=Nc)
    X = np.concatenate((X1,X2))
    y = np.zeros(len(X))
    y[Nc:] = 1
    return X, y

# test the function 
X, y = gen_gaussian()
plt.figure()
plt.scatter(X[y==0., 0], X[y==0, 1], color='r')
plt.scatter(X[y==1, 0], X[y==1, 1], color='b')

In [ ]:
class logreg(): 
    def __init__(self, lr:int=1e-4, max_iter:int=1000, p_stop:float=1e-3): 
        self.lr = lr 
        self.max_iter = max_iter
        self.weights = None 
        self.bias = 0.
        self.p_stop = p_stop
        self.cross_entropy = [] 

    def logistic(self, x:np.ndarray): 
        """
        logistic function: only for a single data point 
        """
        return 1.0/(1+np.exp(-(self.bias + np.dot(self.weights, x))))
    
    def fit(self, X, y): 
        N,d = X.shape
        self.weights = np.zeros(d)

        for t in range(self.max_iter): 
            # do 
            w_old = self.weights
            for i in np.random.permutation(N): 
                gw = self.logistic(X[i])
                self.bias -= self.lr*(gw - y[i])
                self.weights -= self.lr*(gw - y[i])*X[i]
            
            prob = np.array([self.logistic(X[i]) for i in range(len(X))])
            
            self.cross_entropy.append(-np.log(prob[y==1]).sum() - np.log(1-prob[y==0]).sum())

            
            
                
    def predict(self, X:np.ndarray): 
        yhat = np.zeros(len(X))
        phat = np.zeros(len(X))

        for i in range(len(X)): 
            phat[i] = self.logistic(X[i])
            yhat[i] = 1.0*(phat[i]>=0.5)
        return yhat, phat 

# test the function 
X, y = gen_gaussian()

xx, yy = np.meshgrid( np.linspace(-5, 5, 100), np.linspace(-5, 5, 100))
Xt = np.vstack((xx.reshape((np.prod(xx.shape))), yy.reshape((np.prod(yy.shape))) )).T

mdl = logreg(max_iter=100)
mdl.fit(X, y)
yhat, phat = mdl.predict(Xt)

# test the function 
plt.figure()
plt.scatter(Xt[yhat==0., 0], Xt[yhat==0, 1], color='r')
plt.scatter(Xt[yhat==1, 0], Xt[yhat==1, 1], color='b')
plt.title('Testing Data (Predicted Labels)')

plt.figure()
plt.scatter(X[y==0., 0], X[y==0, 1], color='r')
plt.scatter(X[y==1, 0], X[y==1, 1], color='b')
plt.title('Training Data (True Labels)')

xx, yy = np.meshgrid( np.linspace(-5, 5, 100), np.linspace(-5, 5, 100))
Xt = np.vstack((xx.reshape((np.prod(xx.shape))), yy.reshape((np.prod(yy.shape))) )).T
yhat, phat = mdl.predict(Xt)

plt.figure()
plt.pcolor(xx, yy, phat.reshape(100,100))
plt.colorbar()
plt.title('$P(\omega=1|x)$')

In [ ]:
lrs = [0.1, 0.01, 1e-3, 1e-5]
mdls = []
# test the function 
X, y = gen_gaussian()

plt.figure()
for lr in lrs: 
    mdl = logreg(max_iter=100, lr=lr)
    mdl.fit(X, y)
    plt.plot(mdl.cross_entropy, label=''.join(['LR=', str(lr)]))

plt.xlabel('Epoch')
plt.ylabel('Cross Entropy')
plt.legend()

In [ ]:
# set the bandwidth for the kernel density estimator 
s = .01

# radial basis function:
def gauss(x:np.ndarray, m:np.ndarray, s:float): 
    return np.exp(-np.linalg.norm(x-m, axis=1)**2/s)

# generate data for "training" the density estimator 
X, y = gen_cb()
N = len(X)

# generate the mesh grid 
xx, yy = np.meshgrid( np.linspace(0, 1, 100), np.linspace(0, 1, 100))
Xt = np.vstack((xx.reshape((np.prod(xx.shape))), yy.reshape((np.prod(yy.shape))) )).T

# init the probabilities. note this will not work for higher dimensional data 
# or more classes 
pxw = np.zeros(Xt.shape)
pwx = np.zeros(Xt.shape)

# separate out the data by classes & measure the prior probabilities 
Xa, Xb = X[y==1], X[y==2]
pw = [(y==1).sum()/len(y), (y==2).sum()/len(y)]


for i in range(len(Xt)): 
    # get the likelihoods of each class 
    pxw[i,0] = gauss(Xt[i], Xa, s).sum()/len(Xa)
    pxw[i,1] = gauss(Xt[i], Xb, s).sum()/len(Xb)
    # get the posteriors 
    pwx[i,0] = pxw[i,0]*pw[0]/(pxw[i,0]*pw[0] + pxw[i,1]*pw[1])
    pwx[i,1] = pxw[i,1]*pw[1]/(pxw[i,0]*pw[0] + pxw[i,1]*pw[1])


plt.figure()
plt.pcolor(xx, yy, pwx[:,0].reshape(100,100))
plt.colorbar()
plt.title('$P(\omega=red|x)$')

plt.figure()
plt.pcolor(xx, yy, pxw[:,0].reshape(100,100))
plt.colorbar()
plt.title('$P(x|\omega=red)$')